<a href="https://colab.research.google.com/github/Carnoustie/DeepLearningClassKTH/blob/main/resnet_n_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from tensorflow.keras import datasets, layers, models, regularizers, preprocessing, optimizers,Model, Input
from tensorflow.keras.optimizers import schedules, SGD
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Dense, Flatten, Conv2D, Lambda,	Input, BatchNormalization, Activation
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [6]:
(trainX,trainLabels), (testX,testLabels) = datasets.cifar10.load_data()

#Map pixel values from [0, 255}  to -> [0, 1]
#trainX, testX = trainX/255.0 , testX/255.0

#testLabels = to_categorical(testlabels)


trainMean = np.mean(trainX,0)
trainDev = np.std(trainX,0)
trainX = trainX-trainMean
trainX = trainX/trainDev

testX = trainX-trainMean
testX = trainX/trainDev


print("Sanity Check:\n\nmean ", np.mean(trainX),  "\n\nvar: ", np.std(trainX))



Sanity Check:

mean  7.94919685631612e-18 

var:  0.9999999999999998


In [7]:
trainLabels = tf.keras.utils.to_categorical(trainLabels)
testLabels = tf.keras.utils.to_categorical(testLabels)




In [8]:

num_classes = 10
input_shape = (32,32,3)
model_input = Input(shape=input_shape)

def stack(input, filter,n):

    x = input
    if(filter==16):
      xx = layers.Conv2D(filter, (3,3),  strides=(1, 1),padding='same', kernel_initializer='he_normal')(x)
      xx = layers.BatchNormalization(axis=3)(xx)
      xx = layers.Activation('relu')(xx)
      xx = layers.Conv2D(filter, (3,3),  strides=(1, 1),padding='same', kernel_initializer='he_normal')(xx)
      xx = layers.BatchNormalization(axis=3)(xx)
      x = layers.Add()([xx,x]);
      x = layers.Activation('relu')(x)
    else:
      xx = layers.Conv2D(filter, (3,3),  strides=(2, 2),padding='same', kernel_initializer='he_normal')(x)
      xx = layers.BatchNormalization(axis=3)(xx)
      xx = layers.Activation('relu')(xx)
      xx = layers.Conv2D(filter, (3,3),  strides=(1, 1),padding='same', kernel_initializer='he_normal')(xx)
      xx = layers.BatchNormalization(axis=3)(xx)
#      x = layers.Conv2D(32, (1,1), strides=(2, 2))(x)
      x = layers.Lambda(lambda x: tf.pad(x[:, ::2, ::2, :], tf.constant([[0, 0,], [0, 0], [0, 0], [filter//4, filter//4]]), mode="CONSTANT"))(input)
      x = layers.Add()([xx,x])
      x = layers.Activation('relu')(x)
    for i in range(1,n):
      x = layers.Conv2D(filter, (3,3),  strides=(1, 1),padding='same', kernel_initializer='he_normal')(x)
      x = layers.BatchNormalization(axis=3)(x)
      x = layers.Activation('relu')(x)
      xx = layers.Conv2D(filter, (3,3), strides=(1, 1),padding='same', kernel_initializer='he_normal')(x)
      xx = layers.BatchNormalization(axis=3)(xx)
      x = layers.Add()([xx,x])
      x = layers.Activation('relu')(x)
    return x


def resnet(model_input,n):
    
    x = layers.Conv2D(16, kernel_size = (3,3), kernel_initializer='he_normal', padding='same', strides=(1, 1))(model_input)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)

    for k in [16,32,64]:
      x = stack(x,k,n)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Flatten()(x)
    #x = layers.Dense(units = 1000,activation ='relu')(x)
    x = layers.Dense(units = 10, activation ='softmax')(x)
    model = Model(inputs=model_input, outputs=x, name='ResNet')
    return model

#instantiate the model
#resNetModel = resnet(model_input,3)

#display model summary
#resNetModel.summary() 

In [ ]:
LearningRate = optimizers.schedules.CosineDecay(0.1, 100*trainX.shape[0],0.001)

# Learning rate config per the He et al. paper
boundaries = [32000, 48000]
values = [0.1, 0.01, 0.001]
lr_schedule = optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

optimizer_momentum = 0.9
optimizer_additional_metrics = ["accuracy"]
opt = SGD(learning_rate=lr_schedule, momentum=optimizer_momentum)


#opt = SGD(LearningRate, momentum=0.9)
#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#history = model.fit(trainX, trainLabels, epochs=10, validation_data=(testX, testLabels))
generator = preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
batches = generator.flow(trainX, trainLabels, batch_size=64)

def trainModel(optimizer,model,numEpochs):
  model.compile(optimizer=opt,
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

  history = model.fit(batches, steps_per_epoch=trainX.shape[0]/64,epochs=numEpochs,
                      validation_data=(testX, testLabels))
  historyTrain = history.history['accuracy']
  historyVal = history.history['val_accuracy']
  return historyTrain, historyVal

resnet20 = resnet(model_input,3)
# resnet32 = resnet(model_input,5)
# resnet44 = resnet(model_input,7)
# resnet56 = resnet(model_input,9)
# resnet110 = resnet(model_input,18)


resnet20.summary() 
#resnet44.summary() 
#resnet110.summary() 


trainHistory20, testHistory20 = trainModel(opt,resnet20,20)
#trainHistory32, testHistory32 = trainModel(opt,resnet32,2)
#trainHistory44, testHistory44 = trainModel(opt,resnet44,100)
#trainHistory56, testHistory56 = trainModel(opt,resnet56,2)
#trainHistory110, testHistory110 = trainModel(opt,resnet110,100)







Model: "ResNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]'

In [ ]:
#plt.plot(history.history['accuracy'], label='accuracy')
#plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
#plt.xlabel('Epoch')
#plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
#plt.legend(loc='lower right')

#test_loss, test_acc = model.evaluate(testX,  testLabels, verbose=2)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch iteration')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.savefig('34_layer_vgg.png')
testLoss, testAccuracy = resNetModel.evaluate(testX,testLabels, verbose=2)

In [ ]:
plt.plot(saveThickTrain, label='training error 37 layers')
plt.plot(backupVgg19Train, label='training error VGG-19 (19 layers)')
plt.xlabel('Epoch iteration')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.savefig('deep_shallow_Compare.png')
testLoss, testAccuracy = model.evaluate(testX,testLabels, verbose=2)